In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

from tqdm import trange

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, PowerTransformer, MinMaxScaler
from sklearn.metrics import log_loss

from tmpnn import Regression, Classification, L1, L2, Lyapunov1

In [2]:
def balanced_log_loss(y_true, y_pred):
    N0 = np.sum(1 - y_true)
    N1 = np.sum(y_true)

    p0 = np.clip(1 - y_pred, 1e-15, 1 - 1e-15)
    p1 = np.clip(y_pred, 1e-15, 1 - 1e-15)

    log_loss_0 = -np.sum((1 - y_true) * np.log(p0)) / N0
    log_loss_1 = -np.sum(y_true * np.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2

import tensorflow.keras.backend as K
def blloss(y_true, y_pred):
    y_pred=y_pred[:,-1]
    y_true=y_true[:,-1]
    
    N0 = K.sum(1 - y_true)
    N1 = K.sum(y_true)

    p0 = K.clip(1 - y_pred, 1e-15, 1 - 1e-15)
    p1 = K.clip(y_pred, 1e-15, 1 - 1e-15)

    log_loss_0 = -K.sum((1 - y_true) * K.log(p0)) / N0
    log_loss_1 = -K.sum(y_true * K.log(p1)) / N1
    return (log_loss_0 + log_loss_1)/2

In [3]:
xcols=pd.Index(['AB', 'AF', 'AM', 'AX', 'AZ', 'BC', 'BQ', 'CB', 'CC', 'CD ', 'CH', 'CR',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DU', 'DY', 'EE', 'EH',
       'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI', 'FL', 'FR', 'FS', 'GE', 'GI',
       'GL'],
      dtype='object')
print(*xcols)

AB AF AM AX AZ BC BQ CB CC CD  CH CR CU CW  DA DE DF DH DI DL DU DY EE EH EL EP EU FC FD  FE FI FL FR FS GE GI GL


In [20]:
df = pd.read_csv('train.csv')
df = df.fillna(df.mean(numeric_only=True))
X, Y = df.drop(columns=['Id','Class']), np.asfarray(df['Class'])

X['EJ'] = LabelEncoder().fit_transform(X['EJ'])-0.5

# X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.3, random_state=17)

pt = PowerTransformer()
mms = MinMaxScaler((-0.5,0.5))
X = mms.fit_transform(pt.fit_transform(X))
# X_val = mms.transform(pt.transform(X_val))
X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.3, random_state=1)


In [10]:
model = Regression(X.shape[1], 1, 2, 10, 4e-4, regularizer=L1(0.02), scale=0)
history = model.fit(X_tr, Y_tr, epochs=1000, verbose=1, 
                    validation_data=(X_val, Y_val), stop_monitor='loss', patience=200)

print('train', balanced_log_loss(Y_tr, model.predict(X_tr)[:, 0]))
print('valid', balanced_log_loss(Y_val, model.predict(X_val)[:, 0]))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

Epoch 1/1000
2/2 [==============================] - 1s 123ms/step - loss: 1.3301 - val_loss: 2.0465
Epoch 2/1000
2/2 [==============================] - 0s 32ms/step - loss: 1.8542 - val_loss: 1.7877
Epoch 3/1000
2/2 [==============================] - 0s 33ms/step - loss: 1.8118 - val_loss: 1.7268
Epoch 4/1000
2/2 [==============================] - 0s 34ms/step - loss: 1.6212 - val_loss: 1.5416
Epoch 5/1000
2/2 [==============================] - 0s 35ms/step - loss: 1.5857 - val_loss: 1.6695
Epoch 6/1000
2/2 [==============================] - 0s 34ms/step - loss: 1.6051 - val_loss: 1.4444
Epoch 7/1000
2/2 [==============================] - 0s 33ms/step - loss: 1.4535 - val_loss: 1.5485
Epoch 8/1000
2/2 [==============================] - 0s 43ms/step - loss: 1.5188 - val_loss: 1.4978
Epoch 9/1000
2/2 [==============================] - 0s 33ms/step - loss: 1.4666 - val_loss: 1.4561
Epoch 10/1000
2/2 [==============================] - 0s 33ms/step - loss: 1.4427 - val_loss: 1.4662
Epoch 11

KeyboardInterrupt: 

In [21]:
model = Classification(X.shape[1], 1, 2, 10, regularizer=L1(0.05))

model.set_learning_rate(5e-4, metrics=blloss)
history = model.fit(X_tr, Y_tr, validation_data=(X_val, Y_val), 
                    epochs=1000, verbose=1,
                    class_weight=pd.Series(Y_tr).value_counts(normalize=True).to_dict())
model.set_learning_rate(1e-4, metrics=blloss)

history = model.fit(X_tr, Y_tr, validation_data=(X_val, Y_val), 
                    epochs=700, verbose=1,
                    class_weight=pd.Series(Y_tr).value_counts(normalize=True).to_dict())

model.set_learning_rate(1e-5, loss=blloss)
history = model.fit(X_tr, Y_tr, validation_data=(X_val, Y_val), 
                    epochs=500, verbose=1,
                    stop_monitor='val_loss', patience=2,
                    class_weight=pd.Series(Y_tr).value_counts(normalize=True).to_dict())

print('train', balanced_log_loss(Y_tr, model.predict(X_tr)[:, 0]))
print('valid', balanced_log_loss(Y_val, model.predict(X_val)[:, 0]))

Epoch 1/1000
2/2 [==============================] - 1s 226ms/step - loss: 3.0197 - blloss: 9.0783 - val_loss: 5.4518 - val_blloss: 0.7925
Epoch 2/1000
2/2 [==============================] - 0s 44ms/step - loss: 4.7766 - blloss: 1.0455 - val_loss: 4.6119 - val_blloss: 0.6279
Epoch 3/1000
2/2 [==============================] - 0s 46ms/step - loss: 4.9966 - blloss: 1.9182 - val_loss: 5.0487 - val_blloss: 0.5787
Epoch 4/1000
2/2 [==============================] - 0s 39ms/step - loss: 4.7815 - blloss: 0.6133 - val_loss: 4.0945 - val_blloss: 0.5562
Epoch 5/1000
2/2 [==============================] - 0s 37ms/step - loss: 4.2079 - blloss: 0.5904 - val_loss: 4.5680 - val_blloss: 0.5642
Epoch 6/1000
2/2 [==============================] - 0s 43ms/step - loss: 4.5604 - blloss: 0.5954 - val_loss: 4.4768 - val_blloss: 0.5606
Epoch 7/1000
2/2 [==============================] - 0s 40ms/step - loss: 4.4105 - blloss: 0.5909 - val_loss: 4.2325 - val_blloss: 0.5395
Epoch 8/1000
2/2 [======================

In [22]:

model.set_learning_rate(1e-8, loss=blloss)
history = model.fit(X_tr, Y_tr, validation_data=(X_val, Y_val), 
                    epochs=500, verbose=1,
                    stop_monitor='val_loss', patience=2,
                    class_weight=pd.Series(Y_tr).value_counts(normalize=True).to_dict())

print('train', balanced_log_loss(Y_tr, model.predict(X_tr)[:, 0]))
print('valid', balanced_log_loss(Y_val, model.predict(X_val)[:, 0]))

Epoch 1/500
2/2 [==============================] - 1s 149ms/step - loss: 1.2619 - val_loss: 1.3583
Epoch 2/500
2/2 [==============================] - 0s 43ms/step - loss: 1.2603 - val_loss: 1.3582
Epoch 3/500
2/2 [==============================] - 0s 35ms/step - loss: 1.2625 - val_loss: 1.3580
Epoch 4/500
2/2 [==============================] - 0s 35ms/step - loss: 1.2603 - val_loss: 1.3578
Epoch 5/500
2/2 [==============================] - 0s 37ms/step - loss: 1.2583 - val_loss: 1.3576
Epoch 6/500
2/2 [==============================] - 0s 36ms/step - loss: 1.2607 - val_loss: 1.3575
Epoch 7/500
2/2 [==============================] - 0s 45ms/step - loss: 1.2607 - val_loss: 1.3573
Epoch 8/500
2/2 [==============================] - 0s 37ms/step - loss: 1.2602 - val_loss: 1.3571
Epoch 9/500
2/2 [==============================] - 0s 37ms/step - loss: 1.2601 - val_loss: 1.3570
Epoch 10/500
2/2 [==============================] - 0s 37ms/step - loss: 1.2592 - val_loss: 1.3568
Epoch 11/500
2/2 [

# Submit

In [197]:
pd.DataFrame(np.vstack(model.pnn.get_weights())).to_csv('weights.csv',index=False)

In [201]:
weights = pd.read_csv('weights.csv').values
W0 = weights[0]
W1 = weights[1:58]
W2 = weights[58:]
print(W0.shape, W1.shape, W2.shape)

(57,) (57, 57) (3249, 57)


In [207]:
def tm_map(X, weights=None, steps=10, target_dim = 1):
    # W0, W1, W2 = weights
    X = np.hstack((X, np.zeros((X.shape[0], target_dim))))
    for _ in trange(steps):
        X = W0 + np.dot(X, W1) + np.dot((X[:,:,None]*X[:,None,:]).reshape(X.shape[0],-1), W2)
    return X[:,-1]

In [212]:
df = pd.read_csv('train.csv')
df = df.fillna(df.mean(numeric_only=True))
X, Y = df.drop(columns=['Id','Class']), df['Class']
X['EJ'] = LabelEncoder().fit_transform(X['EJ'])-0.5
X = PowerTransformer().fit_transform(X)
X = MinMaxScaler((-0.5,0.5)).fit_transform(X)

In [217]:
print(balanced_log_loss(Y, np.clip((tm_map(X)+0.5),0,1)))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 34.31it/s]

0.23209742188705307
